In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from nmjmc.nn import NeuralMJMCNetwork
from nmjmc.systems import RepulsiveParticles
from nmjmc.sampling import MJMCSampler, MCSampler
from nmjmc.util import draw_config, reference_mapper
import nmjmc
import pickle
from scipy import integrate

In [ ]:
references = np.load("../local_data/particles_reference_configurations.npz")
reference_open = references["reference_open"]
reference_closed = references["reference_closed"]
biased_confs = np.load(
    "../local_data/training_data_particles_biased.npz", allow_pickle=True
)
particle_system = RepulsiveParticles(params=biased_confs["params"].tolist())

In [ ]:
# run sampling on CPU
config = tf.ConfigProto(device_count={"GPU": 0})
sess = tf.Session(config=config)

In [ ]:
dim = 76
t_nodes = [dim] * 3
s_nodes = [dim] * 3
nnicer = 20
leakyRelu = tf.keras.layers.LeakyReLU(alpha=0.1)


def split_cond(x, dimer_split):
    return tf.abs(x[:, 0] - x[:, 2]) < dimer_split


nn = NeuralMJMCNetwork(
    s_nodes,
    t_nodes,
    nnicer,
    layer_activation=leakyRelu,
    split_cond=split_cond,
    system=particle_system,
)
nn.trainable_rnvp(True)

In [ ]:
nn.load_weights("../local_data/pretrained_models/nmjmc_particles_model.h5")

# Sample
----

In [ ]:
# set sampling parameters

N_steps = 15_00  # 0_000
N_confs = 1
temperatures = [0.5, 1.0, 2.0]
noise = 0.01
local_probability = 0.99
remapper = reference_mapper(reference_open, reference_closed)
initial_config = np.array([reference_closed] * N_confs)
stride = 15

In [ ]:
samples = []
for T in temperatures:
    simulate = MJMCSampler(
        nn, local_probability, particle_system.energy, noise=noise, beta=1.0 / T
    )
    sample = simulate.run(
        initial_config,
        N_steps,
        reassign_labels=remapper,
        reporter="notebook",
        verbose=0,
        stride=stride,
    )
    samples.append(sample)
samples = np.array(samples)

In [ ]:
np.save("../local_data/nmjmc_samples_particles", samples)